Geoparquet is the optimised format for tabular data storage based on the parquet popular in the big data community with the roots in the BigTable ancestors.
We say tabular here, as amid the standard allows for nested structured not many implementations support them.
because of that, in the following 

In [ ]:
import xarray as xr
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from shapely.geometry import Point
import json

# Load NetCDF file
ds = xr.open_dataset("your_file.nc")

# Convert to DataFrame
df = ds.to_dataframe().reset_index()

# Extract metadata
metadata = {
    "global_attributes": {k: str(v) for k, v in ds.attrs.items()},
    "variable_attributes": {
        var: {k: str(v) for k, v in ds[var].attrs.items()} for var in ds.data_vars
    }
}



In [ ]:

# Identify latitude and longitude columns
lat_col = next((col for col in df.columns if col.lower() in ['lat', 'latitude']), None)
lon_col = next((col for col in df.columns if col.lower() in ['lon', 'longitude']), None)

if not lat_col or not lon_col:
    raise ValueError("Latitude and Longitude columns not found.")

# Create geometry column
geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Convert to Arrow Table and add metadata
table = pa.Table.from_pandas(gdf)
schema = table.schema.with_metadata({k: json.dumps(v) for k, v in metadata.items()})

# Write to GeoParquet
pq.write_table(table.replace_schema_metadata(schema.metadata), "output.parquet")

In [1]:
import folium

# Coordinates for the polygon
polygon_coords = [[[37.75,48.05],[37.85,48.05],[37.85,47.95],[37.75,47.95],[37.75,48.05]]]

# Folium expects coordinates in (lat, lon) order
polygon_latlon = [[(lat, lon) for lon, lat in ring] for ring in polygon_coords]

# Create a map centered at the first coordinate
m = folium.Map(location=polygon_latlon[0][0], zoom_start=12)

# Add the polygon
folium.Polygon(locations=polygon_latlon[0], color='blue', fill=True, fill_opacity=0.5).add_to(m)

m

In [10]:
!pip install shapely
import requests
import json
from shapely.geometry import shape, Polygon, mapping

# Read STAC item footprint
stac_url = "https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_SM_S_strip_011R_95745_A13803689_8000"
stac_item = requests.get(stac_url).json()
footprint_geom = shape(stac_item['geometry'])

#find footprint of the donieck urbanised area
donback_polygon = Polygon([ (37.75, 47.95), (37.85, 47.95), (37.85, 48.05), (37.75, 48.05), (37.75, 47.95) ])

# Intersection: strip STAC footprint to Donback area
clipped_footprint = footprint_geom.intersection(donback_polygon)

# Add the clipped footprint to the folium map
if not clipped_footprint.is_empty:
    folium.GeoJson(mapping(clipped_footprint), style_function=lambda x: {'color': 'red', 'weight': 3}).add_to(m)

#print clipped footprint in geojson format
print(json.dumps(mapping(clipped_footprint), indent=2))


m


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
{
  "type": "Polygon",
  "coordinates": [
    [
      [
        37.75,
        48.05
      ],
      [
        37.85,
        48.05
      ],
      [
        37.85,
        47.95
      ],
      [
        37.75,
        47.95
      ],
      [
        37.75,
        48.05
      ]
    ]
  ]
}
